# SyftBox SDK Development Testing

This notebook is for testing and validating the SyftBox SDK during development.

In [1]:
%%capture output --no-stderr

# Install the SDK in development mode (run this once)
!pip install nest_asyncio
!pip install -e .

In [2]:
# Import the SDK
import syft_nsai_sdk as sdk
import httpx

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from syft_nsai_sdk.utils.logger import get_logger

logger = get_logger()
logger.success("SyftBox SDK imported successfully")

2025-08-19 19:58:27 - syft_nsai_sdk - SUCCESS - SyftBox SDK imported successfully


In [5]:
# Set up logging
import logging

# Turn off debug logging for your SDK
logging.getLogger('syft_nsai_sdk').setLevel(logging.INFO)

In [6]:
from syft_nsai_sdk import SyftBoxClient
from IPython.display import display, HTML, Markdown

In [7]:
client = SyftBoxClient()
logger.info(f"Syftbox client created: {client}")

2025-08-19 19:58:27 - syft_nsai_sdk.main - INFO - Using email: callis@openmined.org, service URL: https://syftaccounting.centralus.cloudapp.azure.com/
2025-08-19 19:58:27 - syft_nsai_sdk.main - INFO - Accounting configured for callis@openmined.org
2025-08-19 19:58:27 - syft_nsai_sdk.main - INFO - SyftBoxClient initialized for guest@syft.org
2025-08-19 19:58:27 - syft_nsai_sdk - INFO - Syftbox client created: <syft_nsai_sdk.main.SyftBoxClient object at 0x104b9f410>


In [8]:
%%capture output --no-stderr

# Rich model display
import asyncio
models = client.discover_models(health_check="never")
display(HTML(client.list_models()))

2025-08-19 19:58:27 - syft_nsai_sdk.discovery.scanner - INFO - Found 12 models across 251 datasites
2025-08-19 19:58:27 - syft_nsai_sdk.discovery.scanner - INFO - Cached 12 models from 8 owners
2025-08-19 19:58:27 - syft_nsai_sdk.main - INFO - Discovered 12 models (health_check=False)
2025-08-19 19:58:27 - syft_nsai_sdk.main - INFO - Discovered 12 models (health_check=False)


In [9]:
#
# Interactive chat
# STEP 1: List models explicitly
model_details = client.list_models(format="")
print(f"Found the following model details: {len(model_details)}")

2025-08-19 19:58:27 - syft_nsai_sdk.main - INFO - Discovered 12 models (health_check=False)


Found the following model details: 12


In [10]:
# STEP 2: Discover models explicitly 
models = client.discover_models(service_type="chat", tags=["opensource"])
print(f"Found {len(models)} models")

2025-08-19 19:58:27 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for syft-sdk: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-19 19:58:27 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for deepseek-v3: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-19 19:58:27 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for public-tinnyllama: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-19 19:58:27 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for ollama-model: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-19 19:58:27 - syft_nsai_sdk.services.health - INFO - Batch health check completed in 0.03s for 4 models
2025-08-19 19:58:27 - syft_nsai_sdk.main - INFO - Discovered 4 models (health_check=True)


Found 4 models


In [11]:
# model = models[0]
# enabled_services = [s.type.value for s in model.services if s.enabled]
# print(f"{model.name} ({model.owner}) - Tags: {model.tags} - Services: {enabled_services}")

In [12]:
print(f"{'Name':<20} {'Owner':<25} {'Tags':<25} {'Services':<25}")
print("-" * 100)
for model in models:
    enabled_services = [s.type.value for s in model.services if s.enabled]
    tags_str = ", ".join(model.tags[:2])
    services_str = ", ".join(enabled_services)
    print(f"{model.name:<20} {model.owner:<25} {tags_str:<25} {services_str:<25}")

Name                 Owner                     Tags                      Services                 
----------------------------------------------------------------------------------------------------
syft-sdk             callis@openmined.org      chat, opensource          chat                     
deepseek-v3          aggregator@openmined.org  deepseek, opensource      chat, search             
public-tinnyllama    irina@openmined.org       ollama, opensource        chat                     
ollama-model         irina@openmined.org       ollama, opensource        chat                     


In [13]:
# model = client.find_model(model_name="syft-sdk")
model = client.show_model_details(model_name="syft-sdk")
model

2025-08-19 19:58:27 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)


'============================================================\nModel: syft-sdk\n============================================================\nOwner: callis@openmined.org\nSummary: This is syft sdk.\nDescription: This is the syft sdk.\nConfig Status: Active\n\nServices:\n  • Chat: ✅ Enabled (Free)\n  • Search: ❌ Disabled (Free)\n\nTags: chat, opensource, news\n\nPricing: Free\n\nMetadata: /Users/callisezenwaka/SyftBox/datasites/callis@openmined.org/public/routers/syft-sdk/metadata.json'

In [14]:
# public-tinnyllama
model = client.show_model_details(model_name="public-tinnyllama")
model

2025-08-19 19:58:27 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)


'============================================================\nModel: public-tinnyllama\n============================================================\nOwner: irina@openmined.org\nSummary: Public Ollama:tinyllama model hosted for experimentation\nDescription: Try it out!\nConfig Status: Active\n\nServices:\n  • Chat: ✅ Enabled (Free)\n  • Search: ❌ Disabled (Free)\n\nTags: ollama, opensource, tinyllama\n\nPricing: Free\n\nMetadata: /Users/callisezenwaka/SyftBox/datasites/irina@openmined.org/public/routers/public-tinnyllama/metadata.json'

In [15]:
# Check env vars are loaded
import os

email = os.getenv('SYFTBOX_ACCOUNTING_EMAIL')
password = os.getenv('SYFTBOX_ACCOUNTING_PASSWORD')
service_url = os.getenv('SYFTBOX_ACCOUNTING_URL')
print(f"SYFTBOX_ACCOUNTING_EMAIL: {email}")
# print(f"SYFTBOX_ACCOUNTING_PASSWORD: {password}")
print(f"SYFTBOX_ACCOUNTING_URL: {service_url}")

SYFTBOX_ACCOUNTING_EMAIL: callis@openmined.org
SYFTBOX_ACCOUNTING_URL: https://syftaccounting.centralus.cloudapp.azure.com/


In [16]:
# Set up accounting service
# client = SyftBoxClient(auto_setup_accounting=False)

# Sign in to accounting service
await client.setup_accounting(email, password, service_url)

2025-08-19 19:58:27 - syft_nsai_sdk.main - INFO - Using syftbox accounting service URL: https://syftaccounting.centralus.cloudapp.azure.com/
2025-08-19 19:58:28 - syft_nsai_sdk.main - INFO - Accounting configured for callis@openmined.org
2025-08-19 19:58:28 - syft_nsai_sdk.main - INFO - Accounting setup successful


In [17]:
# Check status
print(client.show_accounting_status())
# ✅ Accounting configured
#    Email: user@example.com
#    Balance: $25.50
#    Can use both free and paid models

✅ Accounting configured
   Email: callis@openmined.org
   Balance: $20.0
   Can use both free and paid models


In [16]:
print(await client.get_account_info())

{'email': 'callis@openmined.org', 'balance': 20.0, 'currency': 'USD'}


In [18]:
# STEP 3: Pick and use a specific model 
# Pick and use a specific model  
# chosen_model = model_details[0]
# models = client.list_models(format="json")
response = await client.chat(
    model_name="syft-sdk",
    owner="callis@openmined.org", 
    prompt="Hello! Testing the API",
    temperature=0.7,
    max_tokens=200
)

2025-08-19 19:58:44 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)


In [19]:
print(f"Response:\n{response}")

Response:
ChatResponse(id='67ce5e33-0bce-4013-a7c3-5b5bbe504a9e', model='syft-sdk', message=ChatMessage(role='assistant', content="{'request_id': 'b2e2c3c6-da70-47d5-8a76-84ff0c1fb8f6', 'data': {'message': {'body': {'detail': 'Chat completion failed'}, 'created': '2025-08-19T18:58:44.979053Z', 'expires': '2025-08-20T18:58:44.895734Z', 'headers': {'content-length': '35', 'content-type': 'application/json'}, 'id': 'b2e2c3c6-da70-47d5-8a76-84ff0c1fb8f6', 'method': '', 'sender': 'callis@openmined.org', 'status_code': 500, 'url': 'syft://callis@openmined.org/app_data/syft-sdk/rpc/chat'}}}", name=None), usage=ChatUsage(prompt_tokens=0, completion_tokens=0, total_tokens=0), cost=None, provider_info=None)


In [20]:
# STEP 3: Pick and use a specific model 
# Pick and use a specific model  
# chosen_model = model_details[0]
# models = client.list_models(format="json")
response = await client.chat(
    model_name="public-tinnyllama",
    owner="irina@openmined.org", 
    prompt="Hello! Testing the API",
    temperature=0.7,
    max_tokens=200
)

2025-08-19 19:58:53 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)


PollingTimeoutError: RPC_ERROR: Polling timed out after 20/20 attempts

In [21]:
print(f"Response:\n{response}")

Response:
ChatResponse(id='67ce5e33-0bce-4013-a7c3-5b5bbe504a9e', model='syft-sdk', message=ChatMessage(role='assistant', content="{'request_id': 'b2e2c3c6-da70-47d5-8a76-84ff0c1fb8f6', 'data': {'message': {'body': {'detail': 'Chat completion failed'}, 'created': '2025-08-19T18:58:44.979053Z', 'expires': '2025-08-20T18:58:44.895734Z', 'headers': {'content-length': '35', 'content-type': 'application/json'}, 'id': 'b2e2c3c6-da70-47d5-8a76-84ff0c1fb8f6', 'method': '', 'sender': 'callis@openmined.org', 'status_code': 500, 'url': 'syft://callis@openmined.org/app_data/syft-sdk/rpc/chat'}}}", name=None), usage=ChatUsage(prompt_tokens=0, completion_tokens=0, total_tokens=0), cost=None, provider_info=None)


In [ ]:
response = await client.chat(
    model_name="public-tinnyllama",
    owner="irina@openmined.org", 
    prompt="Hello! Testing the API",
    temperature=0.7,
    max_tokens=200
)

print(f"Response:\n{response}")
# syft://irina@openmined.org/app_data/public-tinnyllama/rpc/chat
# syft://irina@openmined.org/app_data/public-tinnyllama/rpc/chat

## 1. Check SDK Status

In [3]:
# Check if SDK is properly configured
print("📊 Checking SDK status...")
status_info = sb.status()

logger.success(f"🔐 Authenticated: {sb.is_authenticated}")
if sb.current_user:
    logger.info(f"👤 Current user: {sb.current_user}")

2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO - SyftBox SDK Status:
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO - SyftBox SDK Status:
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO -    Config Exists: Yes
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO -    Config Exists: Yes
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO -    Authenticated: Yes
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO -    Authenticated: Yes
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO -    Auth Manager: Yes
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO -    Auth Manager: Yes
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO -    User: callis@openmined.org
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO -    User: callis@openmined.org
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO -    Server: https://syftbox.net
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO -    Server: https://syftbox.net
2025-08-15 15:09:05 - syft_nsai_sdk.sdk - INFO -    Data Dir: /Users/callisezenwaka/SyftBox
2025-08-15 15:09:05 - syft_n

📊 Checking SDK status...


## 2. Discover Models

## 3. Filter Models by Tags

In [5]:
# Search for models with specific tags
test_tags = ["news", "AI", "test", "chat", "ollama", "nlp", "search", "chat"]

for tag in test_tags:
    print(f"\nLooking for '{tag}' models...")
    tagged_models = sb.find_models(tags=[tag])
    print(f"Found {len(tagged_models)} models with '{tag}' tag")
    
    if tagged_models:
        for model in tagged_models:
            print(f"  - {model.name} by {model.owner} | Services: {model.services}")


Looking for 'news' models...
Found 2 models with 'news' tag
  - the-city by speters@thecity.nyc | Services: ['search']
  - ionesio-router by ionesio@openmined.org | Services: ['search']

Looking for 'AI' models...
Found 1 models with 'AI' tag
  - shubham-router by shubham@openmined.org | Services: ['search']

Looking for 'test' models...
Found 1 models with 'test' tag
  - callis by callis@openmined.org | Services: ['chat']

Looking for 'chat' models...
Found 1 models with 'chat' tag
  - claude-sonnet-3.5 by aggregator@openmined.org | Services: ['chat', 'search']

Looking for 'ollama' models...
Found 2 models with 'ollama' tag
  - public-tinnyllama by irina@openmined.org | Services: ['chat']
  - ollama-model by irina@openmined.org | Services: ['chat']

Looking for 'nlp' models...
Found 0 models with 'nlp' tag

Looking for 'search' models...
Found 0 models with 'search' tag

Looking for 'chat' models...
Found 1 models with 'chat' tag
  - claude-sonnet-3.5 by aggregator@openmined.org | S

## 4. Test Model Interaction

In [10]:
# Test model interaction if models are available
if all_models:
    test_model = all_models[0]
    print(f"🤖 Testing interaction with: {test_model.name}")
    print(f"   Owner: {test_model.owner}")
    print(f"   Services: {test_model.services}")
    print(f"   Tags: {test_model.tags}")
    print(f"   Status: {test_model.status}")
    
    # Test chat if available
    if "chat" in test_model.services:
        print("\n💬 Testing chat service...")
        try:
            response = test_model.chat(
                "Hello! This is a test message from the development notebook.",
                temperature=0.7,
                max_tokens=50
            )
            print(f"✅ Chat Response: {response}")
        except Exception as e:
            logger.error(f"❌ Chat failed: {e}")
    
    # Test search if available
    if "search" in test_model.services:
        print("\n🔍 Testing search service...")
        try:
            results = test_model.search("test query")
            logger.success(f"✅ Search Results: {results[:200]}...")
        except Exception as e:
            logger.error(f"❌ Search failed: {e}")
else:
    print("⚠️ No models available for testing")

🤖 Testing interaction with: shubham-router
   Owner: shubham@openmined.org
   Services: ['search']
   Tags: ['AI']
   Status: available

🔍 Testing search service...


2025-08-14 18:30:37 - syft_nsai_sdk - ERROR - ❌ Search failed: Failed to call service 'search': RPC request failed with status 401: {"code":"E_AUTH_INVALID_CREDENTIALS","error":"invalid access token: token signature is invalid: signature is invalid"}



## 5. Test Different Owner Filters

In [7]:
# Test filtering by specific owners
if all_models:
    unique_owners = list(set(model.owner for model in all_models))
    print(f"📊 Found {len(unique_owners)} unique model owners:")
    
    for owner in unique_owners[:3]:  # Test first 3 owners
        print(f"\n👤 Models by {owner}:")
        owner_models = sb.find_models(owners=[owner])
        for model in owner_models:
            print(f"  - {model.name} | {model.services}")

📊 Found 7 unique model owners:

👤 Models by callis@openmined.org:
  - callis | ['chat']

👤 Models by irina@openmined.org:
  - public-tinnyllama | ['chat']
  - openmined-about | ['search']
  - ollama-model | ['chat']

👤 Models by shubham@openmined.org:
  - shubham-router | ['search']


## 6. Performance Testing

In [8]:
import time

# Test discovery performance
print("⏱️ Performance Testing...")

start_time = time.time()
models = sb.get_models()
discovery_time = time.time() - start_time

print(f"📈 Model Discovery: {discovery_time:.2f}s for {len(models)} models")

if models:
    # Test individual model lookup performance
    test_model = models[0]
    
    start_time = time.time()
    found_model = sb.get_model(test_model.name, owner=test_model.owner)
    lookup_time = time.time() - start_time
    
    print(f"🔍 Model Lookup: {lookup_time:.2f}s")
    print(f"✅ Found: {found_model.name if found_model else 'Not found'}")

⏱️ Performance Testing...
📈 Model Discovery: 0.01s for 10 models
🔍 Model Lookup: 0.00s
✅ Found: shubham-router


## 7. Error Handling Tests

In [11]:
# Test error handling
print("Testing error handling...")

# Test non-existent model
try:
    fake_model = sb.get_model("non-existent-model-12345")
    print(f"Non-existent model result: {fake_model}")
except Exception as e:
    logger.error(f"❌ Expected error for non-existent model: {e}")

# Test invalid filters
try:
    empty_results = sb.find_models(tags=["non-existent-tag-12345"])
    print(f"✅ Invalid tag search returned {len(empty_results)} results (expected 0)")
except Exception as e:
    logger.error(f"❌ Unexpected error with invalid tags: {e}")

print("✅ Error handling tests completed")

Testing error handling...
Non-existent model result: None
✅ Invalid tag search returned 0 results (expected 0)
✅ Error handling tests completed


## 8. Development Notes

Use this section to test new features or debug issues:

In [ ]:
# Scratch space for development testing
print("🔧 Development testing area")

# Add your test code here
# Example:
# models = sb.find_models(name="specific-model")
# print(f"Specific search: {len(models)} results")